In [1]:
# Chase Abram
using Pkg
# Pkg.add("DataFrames")
using CSV
using DataFrames
using ForwardDiff
using LinearAlgebra
using Optim

using Random

# Load data
df = DataFrame()
df = CSV.read("psetOne.csv", DataFrame)

,Market,Constant,Price,EngineSize,SportsBike,Brand2,Brand3,z1,z2
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Float64
1,1,1,1.55471,3.0,1,0,0,0.237685,0.0631155
2,1,1,1.55981,5.0,1,0,0,0.00203326,0.186073
3,1,1,1.85719,3.0,1,1,0,0.16455,0.0333083
4,1,1,1.99544,5.0,1,0,1,0.485538,0.117025
5,1,1,4.01799,7.5,1,0,1,0.582503,0.368032
6,1,1,4.14367,10.0,1,0,1,0.564396,0.442613
7,1,1,2.82341,10.0,0,0,1,0.275784,0.26417
8,1,1,6.55832,12.0,0,0,1,1.18369,0.528303
9,2,1,2.33037,5.0,1,0,0,0.316675,0.217231


In [2]:
# names(df)

# Get market t = 17
df17 = df[df."Market" .== 17, :]

# Store variables for market 17
X = convert(Matrix, df17[:, filter(x -> x in ["Constant", "EngineSize", "SportsBike", "Brand2", "Brand3"], names(df17))])
Br = convert(Matrix, df17[:, filter(x -> x in ["Brand2", "Brand3"], names(df17))])
Br = Bool.(Br)
p_given = df17."Price"
df17

,Market,Constant,Price,EngineSize,SportsBike,Brand2,Brand3,z1,z2
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Float64
1,17,1,0.920128,3.0,1,0,0,0.0276024,0.0282533
2,17,1,1.97453,5.0,1,0,0,0.219283,0.296879
3,17,1,1.94548,3.0,1,1,0,0.150052,0.0497719
4,17,1,1.86088,5.0,1,1,0,0.387576,0.301157
5,17,1,1.98307,6.5,0,0,1,0.0714796,0.033719
6,17,1,3.8985,10.0,1,0,1,0.0523355,0.601297
7,17,1,2.86479,10.0,0,0,1,0.152532,0.0601887


In [64]:

# Shares
function s_logit(p, alpha, beta, X, xi)
    
    # Compute numerators for each s_j
    nums = exp.(X*beta - alpha.*p + xi)
    
    return nums./(1 + sum(nums))
end

# Jacobian of shares
function s_logit_jac(p, alpha, beta, X, xi)
    # Get shares
    s = s_logit(p, alpha, beta, X, xi)
    
    # Jacobian is the outer product times alpha in this case...
    jac = alpha .* s * s'
    
    # ... except for an extra term on the diag
    for j in 1: length(p)
        jac[j,j] -= alpha*s[j]
    end
    
    return jac
end

# Hessian of shares (n by n by n tensor)
function s_logit_hess(p, alpha, beta, X, xi)
    
    # Initialize
    hess = zeros(length(p), length(p), length(p))
    
    # Shares
    s = s_logit(p, alpha, beta, X, xi)

    # Jacobian of shares
    sj = s_logit_jac(p, alpha, beta, X, xi)
    
    # Fill in
    for j in 1:length(p)
        for k in 1:length(p)
            for l in 1:length(p)
                
                if j == k
                    # on diag of jacobian
                    hess[j,k,l] = alpha*sj[l,j]*(2*s[j] - 1)
                else
                    # off diag of jacobian
                    hess[j,k,l] = alpha*(sj[l,j]*s[k] + s[j]*sj[l,k])
                end
            end
        end
    end
    
    return hess
end

# Ownership structure
function Omega_star(br)
    
    # Initialize
    Om = zeros(size(br,1),size(br,1))
    
    for i in 1:size(Om,1)
        for j in 1:size(Om,2)
            if br[i,1] && br[j,1]
                # Brand2
                Om[i,j] = 1
            elseif br[i,2] && br[j,2]
                # Brand3
                Om[i,j] = 1
            elseif (!br[i,1]&& !br[j,1]) && (!br[i,2]&& !br[j,2])
                # Brand1
                Om[i,j] = 1
            end
        end
    end
    
    return Om
end

# Omega in Nevo (2001)
function Omega(p, alpha, beta, X, xi, br)
    return Omega_star(br) .* -s_logit_jac(p, alpha, beta, X, xi)
end

# Fixed point expression (want zero at FP)
function fp_logit(p, alpha, beta, X, xi, br)
    return s_logit(p, alpha, beta, X, xi) - Omega(p, alpha, beta, X, xi, br)*p
end

# (Jacobian of Omega)*p (this is used in the gradient of the FP expression)
function jac_om_p(p, alpha, beta, X, xi, br)
    # Initialize
    jacomp = zeros(length(p), length(p))
    
    # Jacobian of shares
    sj = s_logit_jac(p, alpha, beta, X, xi)
    
    # Hessian of shares
    sh = s_logit_hess(p, alpha, beta, X, xi)
    
    # Ownership
    oms = Omega_star(br)
    
    # Fill in
    for i in 1:length(p)
        for k in 1:length(p)
            jacomp[i,k] = -(sh[i,:,k]' * (oms[i,:] .* p))[1]
        end
    end
    return jacomp
end

# Jacobian of fixed point expression
function J_fp_logit(p, alpha, beta, X, xi, br)
    
    # Jacobian of shares
    sj = s_logit_jac(p, alpha, beta, X, xi)
    
    # (Jacobian of Omega)*p
    jacomp = jac_om_p(p, alpha, beta, X, xi, br)
    
    # Omega
    om = Omega(p, alpha, beta, X, xi, br)
#     println("sj: ", sj)
#     println("jacomp: ", jacomp)
#     println("om: ", om)
    return sj - jacomp - om
end

# Solve for fixed point
function solve_fp_logit(p0, alpha, beta, X, xi, br, tol = 1e-14, maxiter = 1000)
    # Initialize price and update storage
    p = zeros(length(p0),1)
    pnew = zeros(length(p0),1)
    
    p .= p0
    
    # Initialize Jacobian
    J = zeros(length(p0), length(p0))
    
    # Iterator
    it = 0
    
    # Difference in updating
    diff = Inf
    println("p0: ", p0)
    
    # Continue until converged or maximum iterations
    while diff > tol && it < maxiter
#         println("p: ", p)
        
        # Jacobian of FP expression
        J .= J_fp_logit(p, alpha, beta, X, xi, br)
#         println("J: ", J)
#         println("J inv: ", inv(J))
#         println("J under fp: ", J \ fp_logit(p, alpha, beta, X, xi, br))
#         println("size(p): ", size(p))
#         println("size(J under fp): ", size(J \ fp_logit(x, alpha, beta, X, xi, br)))
        
        # Update
        pnew .= p .- J \ fp_logit(p, alpha, beta, X, xi, br)
        it += 1
        
#         if it > 20
#             println("s: ", s_logit(p, alpha, beta, X, xi))
#             println("J: ", J)
#             println("det(J): ", det(J))
#             println("fp_logit: ", fp_logit(p, alpha, beta, X, xi, br))
#         end
        
        println("pnew: ", pnew)
        
        # Get difference
        diff = maximum(abs.(pnew .- p))
        println("diff: ", diff)
        
        p .= pnew
    end
    
    println("Took ", it, " iterations")
    println("final fp_logit: ", fp_logit(p, alpha, beta, X, xi, br))
    return p
end

solve_fp_logit (generic function with 3 methods)

In [67]:
p_init = ones(size(X,1),1) .* 1.0
# p .= p_given
# p[7] = 1
alpha = 3
beta = [1 1 2 -1 1]'


# seed maintains results across runs
rng = MersenneTwister(1234)
xi = randn(rng, size(df17,1)) .* 0.0
# println("xi: ", xi)

# println("p: ", p)
# println("delta: ", X*beta - alpha .* p + xi)

s_logit(p_init, alpha, beta, X, xi)
println("shares: ", s_logit(p_init, alpha, beta, X, xi))
# println("sum shares: ", sum(s_logit(p, alpha, beta, X, xi)))

s_logit_jac(p_init, alpha, beta, X, xi)
# println("shares_jac: ", s_logit_jac(p, alpha, beta, X, xi))

# s_logit_hess(p, alpha, beta, X, xi)

# Omega_star(Br)

# Omega(p, alpha, beta, X, xi, Br)

# fp_logit(p, alpha, beta, X, xi, Br)' * fp_logit(p, alpha, beta, X, xi, Br)
# println("fp_logit: ", fp_logit(p, alpha, beta, X, xi, Br))
# fp_logit(p, alpha, beta, X, xi, Br)

# J_fp_logit(p, alpha, beta, X, xi, Br)

out = solve_fp_logit(p_init, alpha, beta, X, xi, Br)
# println("out: ", out)
# println("eval fp_logit: ", fp_logit(out, alpha, beta, X, xi, Br))

# op = optimize(x -> (fp_logit(x, alpha, beta, X, xi, Br)' * fp_logit(x, alpha, beta, X, xi, Br))[1,1], p, LBFGS())

# pmin = op.minimizer
# println("p min: ", pmin)
# println("min: ", op.minimum)
# println("fp_logit: ", fp_logit(pmin, alpha, beta, X, xi, Br))

shares: [0.00029341911289887894; 0.0021680902857082822; 0.00010794285928225993; 0.0007975958427155957; 0.0035745765708455634; 0.8746700481466118; 0.11837371870450338]
p0: [1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 1.0]
pnew: [-65.602119550885; -65.60211955088516; -65.39394071688459; -65.39394071688459; -32.359217027065924; -32.35921702706593; -32.35921702706593]
diff: 66.60211955088516
pnew: [95.15486327937064; 95.1548632793704; 94.65226822982957; 94.65226822982952; 126.98940089581232; 126.98940089581225; 126.98940089581228]
diff: 160.75698283025565
pnew: [95.48937253853487; 95.48937253853462; 94.98678377734532; 94.98678377734527; 127.32361381041652; 127.32361381041645; 127.32361381041648]
diff: 0.33451554751574974
pnew: [95.82387764934586; 95.82387764934562; 95.32129513202251; 95.32129513202246; 127.65782440404719; 127.65782440404712; 127.65782440404715]
diff: 0.33451135467718984
pnew: [96.1583786410202; 96.15837864101997; 95.65580232354866; 95.6558023235486; 127.99203268893692; 127.9920326889368

pnew: [114.5503310792783; 114.55033107927808; 114.04803938764735; 114.0480393876473; 146.3702540284523; 146.37025402845225; 146.37025402845228]
diff: 0.334316208204541
pnew: [114.88464006712437; 114.88464006712415; 114.38235269774799; 114.38235269774793; 146.70434994500766; 146.7043499450076; 146.70434994500764]
diff: 0.33431331010064014
pnew: [115.21894619928965; 115.21894619928943; 114.71666312681006; 114.71666312681; 147.0384441169726; 147.03844411697256; 147.03844411697258]
diff: 0.33431042906207153
pnew: [115.55324949246634; 115.55324949246612; 115.05097069174847; 115.05097069174842; 147.37253655232033; 147.37253655232027; 147.3725365523203]
diff: 0.3343075649384133
pnew: [115.88754996320056; 115.88754996320034; 115.38527540932948; 115.38527540932942; 147.70662725896938; 147.70662725896932; 147.70662725896935]
diff: 0.33430471758100566
pnew: [116.22184762789408; 116.22184762789387; 115.71957729617239; 115.71957729617233; 148.04071624478428; 148.04071624478422; 148.04071624478425]


diff: 0.33408678796871527
pnew: [149.3056651051808; 149.3056651051806; 148.80371840930167; 148.8037184093016; 181.10812484556627; 181.1081248455662; 181.10812484556624]
diff: 0.33408508489571886
pnew: [149.6397459617905; 149.6397459617903; 149.13780179881476; 149.1378017988147; 181.44207395263714; 181.4420739526371; 181.44207395263712]
diff: 0.334083389513097
pnew: [149.97382514203548; 149.97382514203528; 149.47188350058357; 149.47188350058352; 181.77602192218; 181.77602192217995; 181.77602192217998]
diff: 0.3340817017688096
pnew: [150.30790265342594; 150.30790265342574; 149.80596352219487; 149.80596352219482; 182.1099687583937; 182.10996875839365; 182.10996875839368]
diff: 0.33408002161129957
pnew: [150.64197850342174; 150.64197850342154; 150.1400418711844; 150.14004187118434; 182.44391446545387; 182.44391446545382; 182.44391446545384]
diff: 0.3340783489895216
pnew: [150.97605269943273; 150.97605269943253; 150.47411855503725; 150.4741185550372; 182.7778590475131; 182.77785904751303; 1

pnew: [169.3477839927976; 169.3477839927974; 168.84597145244095; 168.84597145244086; 201.1431910684198; 201.14319106841975; 201.14319106841978]
diff: 0.3339953185354432
pnew: [169.68177603122413; 169.68177603122393; 169.17996545630834; 169.17996545630825; 201.47707863677613; 201.47707863677607; 201.4770786367761]
diff: 0.33399400386738876
pnew: [170.01576676797856; 170.01576676797836; 169.5139581507242; 169.5139581507241; 201.8109652836056; 201.81096528360555; 201.81096528360558]
diff: 0.3339926944158549
pnew: [170.34975620820023; 170.34975620820003; 169.84794954087405; 169.84794954087397; 202.14485101197016; 202.1448510119701; 202.14485101197013]
diff: 0.3339913901498619
pnew: [170.68374435699815; 170.68374435699795; 170.18193963191263; 170.18193963191254; 202.47873582491638; 202.47873582491633; 202.47873582491636]
diff: 0.33399009103857225
pnew: [171.01773121945115; 171.01773121945095; 170.51592842896417; 170.5159284289641; 202.81261972547583; 202.81261972547577; 202.8126197254758]
d

pnew: [196.39731506133865; 196.39731506133845; 195.89563992606008; 195.89563992606; 228.18533658171756; 228.1853365817175; 228.18533658171756]
diff: 0.333903440679876
pnew: [196.73121606821735; 196.73121606821715; 196.22954239167566; 196.22954239167558; 228.51915827546173; 228.51915827546168; 228.51915827546173]
diff: 0.33390246561558
pnew: [197.06511610833908; 197.06511610833888; 196.5634438855594; 196.5634438855593; 228.85297925371975; 228.8529792537197; 228.85297925371975]
diff: 0.33390149388372947
pnew: [197.39901518499386; 197.39901518499366; 196.89734441102672; 196.89734441102664; 229.18679951858658; 229.18679951858653; 229.18679951858658]
diff: 0.3339005254673282
pnew: [197.73291330145486; 197.73291330145466; 197.23124397137607; 197.231243971376; 229.52061907214798; 229.52061907214792; 229.52061907214798]
diff: 0.33389956034935153
pnew: [198.06681046097867; 198.06681046097847; 197.5651425698891; 197.56514256988902; 229.85443791648055; 229.8544379164805; 229.85443791648055]
diff:

LoadError: SingularException(5)

In [50]:
# NM: opam: [0.3720742469890362; 0.3724406061806239; 0.3447723065789802; 
# 0.3424326539927414; 2.4893064284464703; 2.5105275013368717; 4.366510826679992]

# BFGS: opam: [0.37346117252339095; 0.37346116923202516; 5.8690207005673365; 
# 0.34160603728869476; 2.5287477985474984; 2.5287470025674392; 2.528747016448347]

# From Sam: 
# 0.3938563345938473
# 0.3938563345938473
# 0.3496634377039666
# 0.3496634413963176
# 1.6632109730370337
# 1.6632109730370337
# 1.6632541435947288





In [52]:
# function f1(x, y, z) 
#     return (x[1] - 5 - y)^2 + (x[2] - 3 - z)^2
# end

# function g1(storage, x, y, z)
#     storage[1] = 2*(x[1] - 5 - y)
#     storage[2] = 2*(x[2] - 3 - z)
# end

# function g2(x,y,z)
#     out = zeros(2)
#     out[1] = 2*(x[1] - 5 - y)
#     out[2] = 2*(x[2] - 3 - z)
    
#     return out
# end

# function tester()
#     a = 1000
#     b = -1000
#     f(x) = f1(x,a,b)
# #     g!(storage, x) = g1(storage, x, a, b)
#     g!(x) = g2(x,a,b)
    
#     out = basic_newton_zero(f, g!, [-200.0, 1000.0])
    
# #     opt = optimize(f, g!, [100.0, -20.0], LBFGS())
# #     return opt, opt.minimizer, opt.minimum
#     return out
# end

# tester()



In [53]:
p_init = ones(size(X,1),1) .* 1.0
# p_init[1] = 50
# p_init[2] = 30
# p_init[3] = 90
# p_init[4] = -12
# p_init[5] = -40
# p_init[6] = 0
# p_init[7] = 1
s(x) = s_logit(x, alpha, beta, X, xi)
# println(s_logit_jac(p, alpha, beta, X, xi))
ForwardDiff.jacobian(s, p_init)

# ForwardDiff.gradient(fp, p)


# fp(x) = fp_logit_2(x, alpha, beta, X, xi, Br)
# println(fp_logit_2_jac(p, alpha, beta, X, xi, Br))
# ForwardDiff.gradient(fp, p)

fp(x) = fp_logit(x, alpha, beta, X, xi, Br)
J_ex = J_fp_logit(p_init, alpha, beta, X, xi, Br)
println(J_ex)
# println(maximum(abs.(ForwardDiff.jacobian(fp, p) .- J_ex)))
ForwardDiff.jacobian(fp, p_init)
# println(inv(J_ex))

[0.0008727277308654773 -7.605723129685889e-6 -1.8863166935850017e-7 -1.3938099869248953e-6 -6.246622984529345e-6 -0.0015284982482105759 -0.00020685974334824474; -7.605723129685891e-6 0.006400040770708964 -1.3938099869248953e-6 -1.0298940184637844e-5 -4.6156647661556936e-5 -0.011294159303145172 -0.001528498248210576; -1.89518735976878e-7 -1.4003645719315781e-6 0.00032280922990471746 -1.0317339910446653e-6 -2.308810856478474e-6 -0.0005649473897043272 -7.645731499942015e-5; -1.4003645719315784e-6 -1.0347372380957437e-5 -1.0317339910446657e-6 0.0023786637026706006 -1.7059932940339566e-5 -0.004174427955469709 -0.0005649473897043273; 1.252234578345595e-5 9.252831548416369e-5 4.606713568973342e-6 3.403926499284905e-5 -0.02126277969578708 0.018569136311739884 0.0025130593222085864; 0.0030641169862345484 0.022640932304973393 0.0011272256445798898 0.00832913352395407 0.018569136311739877 -0.6776819407849809 0.6149253414458312; 0.00041468314020216867 0.0030641169862345484 0.00015255340188079274 0

7×7 Array{Float64,2}:
  0.000872728  -7.60572e-6  -1.88632e-7   …  -0.0015285    -0.00020686
 -7.60572e-6    0.00640004  -1.39381e-6      -0.0112942    -0.0015285
 -1.89519e-7   -1.40036e-6   0.000322809     -0.000564947  -7.64573e-5
 -1.40036e-6   -1.03474e-5  -1.03173e-6      -0.00417443   -0.000564947
  1.25223e-5    9.25283e-5   4.60671e-6       0.0185691     0.00251306
  0.00306412    0.0226409    0.00112723   …  -0.677682      0.614925
  0.000414683   0.00306412   0.000152553      0.614925     -0.623419

In [54]:
# Compute probabilities of agent i choosing product j
function pr(delta::AbstractVector{T}, X, sigma, zeta) where T
    
    # Initialize utilities
    ubar = zeros(T, size(zeta,2), size(delta,1))
    
    # Compute utilities
    for i in 1:size(ubar, 1)
        for j in 1:size(ubar, 2)
            ubar[i,j] = delta[j] + X[j,:]' * sigma * zeta[:,i]
        end
    end
    
    # Initialize probabilities
    p = zeros(T, size(ubar,1), size(ubar,2))
    
    # Compute probabilities
    for i in 1:size(p,1)
        for j in 1:size(p,2)
            p[i,j] = exp(ubar[i,j])/(1 + sum(exp.(ubar[i,:])))
        end
    end
    
    return p
end



# Shares
function sHat(delta::AbstractVector{T}, X, sigma, zeta) where T
    
    # Compute probs
    p = pr(delta, X, sigma, zeta)
    
    # Initialize shares
    s = zeros(T, size(delta))
    
    # Compute share
    for j in 1:length(s)
        s[j] = 1/size(zeta,2) * sum(p[:,j])
    end
    
    return s
end


# Test cases
nI = 20
nJ = 3
nN = 5

delta_1 = zeros(3)
X_1 = zeros(3,nN)
sigma_1 = 0.0 .* I(size(X,2))
zeta_1 = zeros(nN, nI)

delta_2 = zeros(3)
delta_2[1] = 40
delta_2[2] = 20
delta_2[3] = 20
X_2 = zeros(nJ,nN)
sigma_2 = 0.0 .* I(size(X,2)).* 0.1
zeta_2 = zeros(nN, nI)

sigma_3 = 0.1 .* I(nN)
delta_3 = zeros(nJ)
X_3a = zeros(nJ,nN)

rng = MersenneTwister(1234)
zeta_3 = randn(rng, nN, nI)

rng = MersenneTwister(1234)
X_3b = randn(rng, size(X_3a))
X_3c = X_3b .* 10
X_3d = abs.(X_3c)


s1 = sHat(delta_1, X_1, sigma_1, zeta_1)
s2 = sHat(delta_2, X_2, sigma_2, zeta_2)
s3a = sHat(delta_3, X_3a, sigma_3, zeta_3)
s3b = sHat(delta_3, X_3b, sigma_3, zeta_3)
s3c = sHat(delta_3, X_3c, sigma_3, zeta_3)
s3d = sHat(delta_3, X_3d, sigma_3, zeta_3)

println("s1: ", s1)
println("s2: ", s2)
println("s3a: ", s3a)
println("s3b: ", s3b)
println("s3c: ", s3c)
println("s3d: ", s3d)

s1: [0.25, 0.25, 0.25]
s2: [0.999999995877693, 2.0611536139418496e-9, 2.0611536139418496e-9]
s3a: [0.25, 0.25, 0.25]
s3b: [0.25396627272710803, 0.23546917816783988, 0.2598787383914339]
s3c: [0.302761733713778, 0.18340110893056388, 0.3631032237834327]
s3d: [0.20718104155891687, 0.1964933641693537, 0.23876547586390232]


In [55]:
# Jacobian of shares
function sHat_jac(delta, X, sigma, zeta)
    # Get probabilities
    p = pr(delta, X, sigma, zeta)
    
    # Get shares
    s = sHat(delta, X, sigma, zeta)
    
    # Initialize Jacobian
    sj = zeros(size(delta,1), size(delta,1))
    
    # Fill in
    for j in 1:size(sj,1)
        for k in 1:size(sj,2)
            if j == k
                sj[j,k] = 1/size(zeta,2) * sum(p[:,j] .* (1 .- p[:,j]))
            else
                sj[j,k] = 1/size(zeta,2) * sum(-p[:, j] .* p[:, k])
            end
        end
    end
    
    return sj
end

# Jacobian of log of shares
function sHat_log_jac(delta, X, sigma, zeta)
    # shares
    s = sHat(delta, X, sigma, zeta)
    
    # jacobian of shares
    sj = sHat_jac(delta, X, sigma, zeta)
    
    # initialize jacobian of log of share
    slj = zeros(size(delta,1), size(delta,1))
    
    # Fill in
    for j in 1:size(slj, 1)
        for k in 1:size(slj,2)
            slj[j,k] = sj[j,k]/s[k]
        end
    end
    
    return slj
end

s1_jac = sHat_jac(delta_1, X_1, sigma_1, zeta_1)
println("s1_jac: ", s1_jac)
s1_log_jac = sHat_log_jac(delta_1, X_1, sigma_1, zeta_1)
println("s1_log_jac: ", s1_log_jac)

s1_jac: [0.1875 -0.0625 -0.0625; -0.0625 0.1875 -0.0625; -0.0625 -0.0625 0.1875]
s1_log_jac: [0.75 -0.25 -0.25; -0.25 0.75 -0.25; -0.25 -0.25 0.75]


In [59]:
function sHat_inv(s, X, sigma, zeta, tol = 1e-14, maxiter = 1e5)
    
    # Initialize
    delta = zeros(size(X,1))
    shat = zeros(size(delta))
    
    diff = maximum(log.(s) - log.(sHat(delta, X, sigma, zeta)))
    it = 0
    
    while diff > tol && it < maxiter
#         println("it: ", it)
        
        shat = sHat(delta, X, sigma, zeta)
#         println("shat: ", shat)
        
#         if diff > 1e-3
#             println("Contraction")
            inc = log.(s) - log.(shat)
#         else
#             println("Newton")
            
#             slj = sHat_log_jac(delta, X, sigma, zeta)
# #             println("slj: ", slj)
#             inc = -slj \ log.(shat)
#         end
        
        delta += inc
        
        diff = maximum(abs.(inc))
#         println("diff: ", diff)
        it += 1
    end
    println("Loop exited")
    
    println("it: ", it)
    println("diff: ", diff)
    
    
    return delta
end

println("s1_inv: ", sHat_inv(s1, X_1, sigma_1, zeta_1))
println("s2_inv: ", sHat_inv(s2, X_2, sigma_2, zeta_2))
println("s3a_inv: ", sHat_inv(s3a, X_3a, sigma_3, zeta_3))
# println("s3b_inv: ", sHat_inv(s3b, X_3b, sigma_3, zeta_3))
# println("s3c_inv: ", sHat_inv(s3c, X_3c, sigma_3, zeta_3))
# println("s3d_inv: ", sHat_inv(s3d, X_3d, sigma_3, zeta_3))

Loop exited
it: 0
diff: 0.0
s1_inv: [0.0, 0.0, 0.0]
Loop exited
it: 100000
diff: 9.999750006399497e-6
s2_inv: [11.5129554604045, -8.487044539595498, -8.487044539595498]
Loop exited
it: 0
diff: 0.0
s3a_inv: [0.0, 0.0, 0.0]


In [60]:
delta_1 = ones(3)
delta_2 = 2
delta_3 = 40
X_1 = 10 .* randn(3,nN)
sigma_1 = 0.1 .* I(size(X,2))
zeta_1 = randn(nN, 10000)

shl(x) = log.(sHat(x, X_1, sigma_1, zeta_1))
println(sHat_log_jac(delta_1, X_1, sigma_1, zeta_1))
ForwardDiff.jacobian(shl, delta_1)

[0.312732027888357 -0.1537762920452742 -0.19599780799623104; -0.13365590331755065 0.4036926094101785 -0.15359595212321064; -0.14794550874228118 -0.13339255188442317 0.4890626685181003]


3×3 Array{Float64,2}:
  0.312732  -0.133656  -0.147946
 -0.153776   0.403693  -0.133393
 -0.195998  -0.153596   0.489063